In [1]:
from __future__ import division
from pyomo.environ import *
import numpy as np
import pandas as pd
import time
from functools import reduce

In [3]:
products=2
weeks=3
coef_p1 = [-6,0.01,1,-0.003,23]
coef_p2 = [1.2,-0.004,-5,0.02,20]
coef = [coef_p1,coef_p2]

base_price_1=[10,10,10]
base_price_2=[10,10,10]
base_price = [base_price_1, base_price_2]

edlp_p1=[9.5,10,10]
edlp_p2 =[10,10,9.6]
edlp = [edlp_p1,edlp_p2]

tpr_p1=[0,5,15]
tpr_p2 =[15,5,0]
tpr = [tpr_p1,tpr_p2]

flag_p1=[1,0,0]
flag_p2=[0,0,1]
flag = [flag_p1,flag_p2]

In [4]:
def cal_sales_prior(edlp,tpr,flag,coef,j):
    unit_sales = []
    for i in  range(weeks):
        edlp_sum = 0
        tpr_sum = 0
        total = 0
        cc= 0
        for p in range(products):
            if p==j:
                z = 0
                edlp_sum+= (np.log( edlp[p][i])*coef[cc])* z
                tpr_sum+= (np.log(base_price[p][i])*coef[cc]+ tpr[p][i]*z)*(1- z)
            else:
                edlp_sum+= (np.log( edlp[p][i])*coef[cc])* flag[p][i]
                tpr_sum+= (np.log(base_price[p][i])*coef[cc]+ tpr[p][i]*coef[cc+1])*(1- flag[p][i])
                
            cc+=2
            
        unit_sales.append(np.exp(edlp_sum+ tpr_sum +coef[4]))                 
    return unit_sales

def cal_sales(edlp,tpr,flag,coef,j):
    unit_sales = []
    for i in  range(weeks):
#         print(edlp_p1[i],coef[0],flag_p1[i])
        edlp_sum = 0
        tpr_sum = 0
        cc= 0
        for p in range(products):
            edlp_sum+= (np.log( edlp[p][i])*coef[cc])* flag[p][i]
            tpr_sum+= (np.log(base_price[p][i])*coef[cc]+ tpr[p][i]*coef[cc+1])*(1- flag[p][i])
            cc+=2
            
            
        unit_sales.append(np.exp(edlp_sum+ tpr_sum +coef[4]))                 
   
    return unit_sales

def calc_price(edlp,tpr,flag,j):
    price = []
    for i in range(weeks):
        price.append((base_price[j][i]*(1-tpr[j][i]/100)*(1-flag[j][i]))  +  (edlp[j][i]*flag[j][i]))
                   
    return price
                   
def calc_dollar_sales(edlp,tpr,flag,coef, prior=False):
    dollar_sales=[]
    for j in range(products):
        if not prior:
            unit_sales= cal_sales(edlp,tpr,flag,coef[j],j)
            price = calc_price(edlp,tpr,flag,j)
        else:
            unit_sales = cal_sales_prior(edlp,tpr,flag,coef[j],j)
            price = base_price[j]

        dollar_sales_single = [unit_sales[i]*price[i] for i in range(weeks)]
        dollar_sales.append(dollar_sales_single)
    
    return dollar_sales

def calc_total_trade_spent(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    total_trade_spent = []
    for j in range(products):
        unit_sales= cal_sales(edlp_p1, tpr_p1, flag_p1, 
                              edlp_p2, tpr_p2, flag_p2, coef[j])
        price = calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]
        total_trade_spent.append(total_trade_spent_partial)
        
    return total_trade_spent

def calc_edlp_trade_spent(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,coef):
    edlp_trade_spent = []
    flag = [flag_p1,flag_p2]
    for j in range(products):
        unit_sales= cal_sales(edlp_p1, tpr_p1, flag_p1, 
                              edlp_p2, tpr_p2, flag_p2, coef[j])
        price = calc_price(edlp_p1, tpr_p1, flag_p1, edlp_p2, tpr_p2, flag_p2,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]

        edlp_trade_spent_partial = [total_trade_spent_partial[i]*flag[j][i] for i in range(weeks)]
        edlp_trade_spent.append(edlp_trade_spent_partial)
    return edlp_trade_spent


def calculate_lift(dollar_sales, prior_prom_sales):
    lift = []
    for i in range(products):
        for j in range(weeks):
            lift.append(dollar_sales[i][j] - prior_prom_sales[i][j])
    return lift

In [6]:
initial = calc_dollar_sales(edlp,tpr,flag,coef,True)
final = calc_dollar_sales(edlp,tpr,flag,coef)

print(initial)
print(final)
calculate_lift(final, initial)

[[931600.7555572463, 959972.2223984859, 935501.1308398938], [723032.7419486993, 753709.0819777541, 724155.7261574194]]
[[1203959.2996011795, 958731.498194816, 923862.6607326979], [829593.297652488, 791328.4901867423, 852603.3544627768]]


[272358.5440439333,
 -1240.7242036698153,
 -11638.470107195899,
 106560.55570378865,
 37619.40820898814,
 128447.62830535742]